In [1]:
import kagglehub
import os
import shutil

# Define the new destination directory
new_destination = "/dataset"  # Example: change this to your desired path

# Download the dataset to the default location
path = kagglehub.dataset_download(
    "mehantkammakomati/atm-anomaly-video-dataset-atmav"
)

print("Path to dataset files (default location):", path)

# Check if the new directory exists, and create it if it doesn't
if not os.path.exists(new_destination):
    os.makedirs(new_destination)

# Move the downloaded dataset to the new directory
try:
    # List the files in the source path
    files = os.listdir(path)
    
    # Move each file to the new destination
    for file in files:
        source_file = os.path.join(path, file)
        destination_file = os.path.join(new_destination, file)
        shutil.move(source_file, destination_file)
        print(f"Moved '{file}' from '{path}' to '{new_destination}'")
    
    # Remove the now empty source directory
    shutil.rmtree(path)
    print(f"Removed empty directory: '{path}'")
    
    #change the dataset_path variable
    dataset_path = new_destination
    print(f"The dataset path has now been set to '{dataset_path}'")
except FileNotFoundError as e:
    print(f"Error moving dataset: {e}")

c:\Users\rrsan\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  7%|▋         | 22.0M/297M [00:07<01:28, 3.25MB/s]


KeyboardInterrupt: 

In [3]:
import cv2
import torch

In [5]:
import numpy as np
import time
import os
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\rrsan\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:

model = YOLO('yolov8n.pt')  # Using the nano version for efficiency

# Define classes of interest
classes_of_interest = ['person', 'hand', 'weapon', 'card', 'phone', 'wallet']

# Suspicious activity flags
suspicious_activities = {
    'touching_non_screen_area': False,
    'bringing_weapon': False,
    'blocking_camera': False,
}

# Dataset path
dataset_path = r"dataset"
videos_path = os.path.join(dataset_path, "videos")
labels_path = os.path.join(dataset_path, "labels")

100%|██████████| 6.25M/6.25M [00:02<00:00, 2.87MB/s]


In [7]:
# Video writer setup
fourcc = cv2.VideoWriter_fourcc(*'XVID')
recording = False
out = None
last_suspicious_time = 0
record_duration = 10  # seconds



In [8]:

def check_suspicious_activities(detections, frame):
    height, width, _ = frame.shape
    atm_screen_area = (width // 3, height // 3, width * 2 // 3, height * 2 // 3)  # Define ROI for screen
    suspicious_activities.update({'touching_non_screen_area': False, 'bringing_weapon': False, 'blocking_camera': False})

    for detection in detections:
        label = detection["name"]
        x1, y1, x2, y2 = map(int, [detection['xmin'], detection['ymin'], detection['xmax'], detection['ymax']])

        if label == 'hand':
            if not (atm_screen_area[0] < (x1 + x2) / 2 < atm_screen_area[2] and atm_screen_area[1] < (y1 + y2) / 2 < atm_screen_area[3]):
                suspicious_activities['touching_non_screen_area'] = True

        if label == 'weapon':
            suspicious_activities['bringing_weapon'] = True

        if label == 'person' and y1 < height // 4:  # Person covering the camera
            suspicious_activities['blocking_camera'] = True

    return any(suspicious_activities.values())



In [9]:
# Process each video in the dataset
for video_file in os.listdir(videos_path):
    if video_file.endswith(('.mp4', '.avi', '.mov')):  # Check for video file extensions
        video_path = os.path.join(videos_path, video_file)
        cap = cv2.VideoCapture(video_path)

        if not cap.isOpened():
            print(f"Error: Could not open video {video_file}")
            continue

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            results = model(frame)  # Run YOLOv8 model
            detections = results[0].boxes.data.cpu().numpy()  # Get detections

            detection_list = []
            for det in detections:
                x1, y1, x2, y2, conf, cls = det
                label = model.names[int(cls)]
                if label in classes_of_interest:
                    detection_list.append({
                        'name': label,
                        'xmin': x1, 'ymin': y1, 'xmax': x2, 'ymax': y2
                    })
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                    cv2.putText(frame, f"{label}: {conf:.2f}", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            if check_suspicious_activities(detection_list, frame):
                cv2.putText(frame, "DANGER", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                current_time = time.time()
                if not recording:
                    last_suspicious_time = current_time
                    output_filename = f"ATM_alert_{int(time.time())}.avi"
                    out = cv2.VideoWriter(output_filename, fourcc, 20.0, (frame.shape[1], frame.shape[0]))
                    recording = True

            if recording:
                out.write(frame)
                if time.time() - last_suspicious_time > record_duration:
                    recording = Falsqqqe
                    out.release()

            cv2.imshow("ATM Security System", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
        if recording:
            out.release()

print("Processing complete.")



0: 544x640 1 person, 173.7ms
Speed: 5.5ms preprocess, 173.7ms inference, 2.1ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 1 person, 159.5ms
Speed: 6.7ms preprocess, 159.5ms inference, 1.3ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 1 person, 122.8ms
Speed: 3.9ms preprocess, 122.8ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 1 person, 121.6ms
Speed: 6.1ms preprocess, 121.6ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 1 person, 116.5ms
Speed: 3.2ms preprocess, 116.5ms inference, 1.6ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 1 person, 115.6ms
Speed: 3.8ms preprocess, 115.6ms inference, 1.2ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 1 person, 116.6ms
Speed: 4.5ms preprocess, 116.6ms inference, 1.2ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 1 person, 115.7ms
Speed: 3.7ms preprocess, 115.7ms inference, 1.4ms postprocess per image at

KeyboardInterrupt: 